## Chess-Piece YOLOv11n Training - RX 6800 XT

###  Verify ROCm

In [2]:
import torch, os, subprocess, datetime
print("PyTorch:", torch.__version__, "ROCm:", torch.version.hip)
print("GPU:", torch.cuda.get_device_name(0))

PyTorch: 2.4.1+cpu ROCm: None


AssertionError: Torch not compiled with CUDA enabled

Initialise data.yaml

In [2]:
yaml_path = "../Dataset/data.yaml" 

### Train YOLOv11n 320x320

In [4]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")               # nano pretrained
proj_name = "chess_yolo11n"
results = model.train(
    data=yaml_path,
    epochs=150,
    imgsz=320,
    batch=64,               # 6800XT 16 GB handles 64 easily
    device=0,
    optimizer="SGD",
    lr0=0.01,
    weight_decay=5e-4,
    mosaic=0,               # keep geometry clean
    mixup=0,
    exist_ok=False,
    project=proj_name,
    name="exp",
)

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.5.0) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.10.12 torch-2.4.1+rocm6.0 CUDA:0 (AMD Radeon RX 6800 XT, 16368MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../Dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=0, multi_scale=False, name=exp, nbs=64, nms=False, opset=None,

### Evaluate & Print Metrics

In [3]:
import torch, os, subprocess, datetime
from ultralytics import YOLO
model = YOLO(os.path.join("chess_yolo11n", "exp", "weights", "best.pt"))
test_metrics = model.val(data=yaml_path, split='test', save_json=True)
print("mAP@0.5 :", test_metrics.box.map50)
print("mAP@0.5:0.95 :", test_metrics.box.map)
print("Test F1-ish  :", test_metrics.box.f1)   # Ultralytics gives per-class F1
f1_macro = test_metrics.box.f1.mean()
print(f"Test Macro-F1 = {f1_macro:.3f}")


Ultralytics 8.3.215  Python-3.11.9 torch-2.4.1+cpu CPU (AMD Ryzen 5 9600X 6-Core Processor)
YOLO11n summary (fused): 100 layers, 2,584,492 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 476.3184.4 MB/s, size: 21.1 KB)
val: Scanning E:\Robotics_CW2\Dataset\test\labels.cache... 247 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 247/247  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 5.1it/s 3.1s0.1s
                   all        247       5865      0.879      0.789      0.883      0.579
          black-bishop        183        402      0.807      0.729      0.831       0.51
           black-queen        184        475      0.936      0.863      0.934      0.631
          black-knight        188        307      0.862      0.671      0.817      0.512
            black-pawn        174        356      0.867      0.769      0.875       0.59
            black-king        176       

###  Export INT8 TFLite for Pi

In [7]:
onnx_path = model.export(format="onnx", imgsz=320, simplify=True)
tflite_path = model.export(format="tflite", imgsz=320, int8=True,
                           data=yaml_path)   # calibrate with 100 images
print("ONNX :", onnx_path)
print("TFLite :", tflite_path)

Ultralytics 8.3.221 🚀 Python-3.10.12 torch-2.4.1+rocm6.0 CPU (AMD Ryzen 5 9600X 6-Core Processor)

PyTorch: starting from '/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 16, 2100) (5.2 MB)

ONNX: starting export with onnx 1.19.1 opset 19...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 0.7s, saved as '/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best.onnx' (10.0 MB)

Export complete (0.8s)
Results saved to /media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights
Predict:         yolo predict task=detect model=/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best.onnx imgsz=320  
Validate:        yolo val task=detect model=/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best.onnx imgsz=320 data=../Dataset/data.yaml  
Visuali

I0000 00:00:1762800254.251960   15605 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1762800254.252061   15605 single_machine.cc:374] Starting new session
W0000 00:00:1762800254.492473   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762800254.492487   15605 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1762800254.793984   15605 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1762800254.794059   15605 single_machine.cc:374] Starting new session
W0000 00:00:1762800255.010400   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762800255.010415   15605 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1762800255.341461   15605 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1762800255.341542   15605 single_machine.cc:374] 

W0000 00:00:1762800256.888536   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762800256.888549   15605 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1762800256.900233   15605 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
fully_quantize: 0, inference_type: 6, input_inference_type: FLOAT32, output_inference_type: FLOAT32
W0000 00:00:1762800302.600160   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762800302.600174   15605 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
W0000 00:00:1762800347.777491   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1762800347.777504   15605 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
W0000 00:00:1762800437.484147   15605 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_for

TensorFlow SavedModel: export success ✅ 297.9s, saved as '/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best_saved_model' (33.4 MB)

TensorFlow Lite: starting export with tensorflow 2.19.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best_saved_model/best_int8.tflite' (2.7 MB)

Export complete (297.9s)
Results saved to /media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights
Predict:         yolo predict task=detect model=/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best_saved_model/best_int8.tflite imgsz=320 int8 
Validate:        yolo val task=detect model=/media/hotdesk/72864CA1864C6825/Robotics_CW2/Training Code/chess_yolo11n/exp/weights/best_saved_model/best_int8.tflite imgsz=320 data=../Dataset/data.yaml int8 
Visualize:       https://netron.app
ONNX : /media/hotdesk/72864CA1864C6825